# Full Pipeline

## Imports

In [1]:
import os
from PIL import Image
from sahi.model import MmdetDetectionModel
from sahi.predict import get_prediction, get_sliced_prediction, predict
from sahi.utils.cv import visualize_object_predictions
import numpy as np
import pickle
from collections import defaultdict
import pandas as pd

/vol/bitbucket/jrb21/anaconda3/envs/sodb/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Google Earth Image resize

In [4]:
image_sizes = set()

for directory in os.listdir('warwickshire_images'):
    print('dir:',directory)
    for file in os.listdir(os.path.join('warwickshire_images', directory)):
        if file.split('.')[-1] == 'jpg':
            print('----',file)
            image = Image.open(os.path.join('warwickshire_images', directory, file))
            image_sizes.add(image.size) # Output: (400, 267)

print(image_sizes)

dir: 2020
---- 57624_4-21-2020.jpg
---- 73741_4-21-2020.jpg
---- 77206_4-21-2020.jpg
---- 8625_4-21-2020.jpg
---- 73315_4-21-2020.jpg
---- 8625_4-11-2020.jpg
---- 46146_4-21-2020.jpg
---- 57893_4-21-2020.jpg
---- 84027_4-9-2020.jpg
---- 38708_4-11-2020.jpg
---- 89130_4-21-2020.jpg
---- 38708_4-21-2020.jpg
---- 89134_4-9-2020.jpg
---- 49698_4-21-2020.jpg
---- 36022_4-21-2020.jpg
---- 84027_4-21-2020.jpg
---- 7153_4-9-2020.jpg
---- 58312_4-11-2020.jpg
---- 89130_4-9-2020.jpg
---- 56417_4-21-2020.jpg
---- 58312_4-21-2020.jpg
---- 84042_4-21-2020.jpg
---- 73742_4-21-2020.jpg
---- 7153_4-21-2020.jpg
---- 77205_4-21-2020.jpg
---- 84042_4-9-2020.jpg
---- 6142_4-21-2020.jpg
---- 84026_4-21-2020.jpg
---- 84047_4-21-2020.jpg
---- 84047_4-9-2020.jpg
---- 89134_4-21-2020.jpg
---- 58322_4-21-2020.jpg
---- 58322_4-11-2020.jpg
---- 73851_4-21-2020.jpg
---- 38717_4-21-2020.jpg
---- 36505_4-11-2020.jpg
---- 36505_4-21-2020.jpg
dir: 2018
---- 77206_4-5-2018.jpg
---- 56417_4-5-2018.jpg
---- 84047_4-5-201

### Make the directory for the resized images

In [5]:
os.mkdir('warwickshire_images_resized')
for dirname in os.listdir('warwickshire_images'):
    os.mkdir(os.path.join('warwickshire_images_resized', dirname))

### Resize the image

On all the images we see that the 60m scale is 310 pixels long. 

We want a GSD of 0.3m therefore we want 60m = 60/0.3 = 200 pixels.

Therefore we multiply the widths and heights by 200/310

In [6]:
width, height = image.size
new_width, new_height = width * 200/310, height * 200/310
new_width, new_height

(3096.7741935483873, 1927.0967741935483)

In [7]:
for directory in os.listdir('warwickshire_images'):
    # Directorie are the years
    for file in os.listdir(os.path.join('warwickshire_images', directory)):
        if file.split('.')[-1] == 'jpg':
            # The image files
            image = Image.open(os.path.join('warwickshire_images', directory, file))
            image.thumbnail((new_width, new_height))
            image.save(os.path.join('warwickshire_images_resized', directory, file))

In [8]:
image.size

(3096, 1927)

## Inference

### Test function: get_sliced_prediction

In [9]:
detection_model = MmdetDetectionModel(
    model_path= "models-to-test/latest.pth",
    config_path= "mmdet_configs/xview_tood/tood_crop_300_500_cls_cars_trucks_1e-3_new_pipe.py",
    device='cuda' # or 'cpu'
)

load checkpoint from local path: models-to-test/latest.pth


In [32]:
test_img_path = "warwickshire_images_resized/2020/73742_4-21-2020.jpg"
width, height = (400,400)

sliced_pred_result = get_sliced_prediction(
    test_img_path,
    detection_model, 
    slice_width=width, 
    slice_height=height
)

Performing prediction on 60 number of slices.


In [33]:
sliced_pred_result.object_prediction_list[0].bbox.__dict__.items()

dict_items([('minx', 2756), ('miny', 1659), ('maxx', 2770), ('maxy', 1670), ('shift_x', 0), ('shift_y', 0)])

In [34]:
min([item.score.value for item in sliced_pred_result.object_prediction_list])

0.30684584379196167

It appears that the function get_sliced_prediction has a confidence threshold of 0.3.

In [35]:
viz_path = "test_result"
test_img_name = "test5"


visualize_object_predictions(
    image=np.ascontiguousarray(sliced_pred_result.image),
    object_prediction_list=sliced_pred_result.object_prediction_list,
    rect_th=1,
    text_size=0.3,
    text_th=1,
    color=(0, 0, 0),
    output_dir=viz_path,
    file_name=test_img_name + "_sliced_pred_result",
    export_format="png",
)

{'image': array([[[84, 84, 84],
         [91, 91, 91],
         [96, 96, 94],
         ...,
         [19, 20, 22],
         [20, 21, 23],
         [61, 62, 64]],
 
        [[94, 94, 92],
         [92, 92, 90],
         [95, 95, 93],
         ...,
         [ 4,  5,  7],
         [ 7,  8, 10],
         [49, 50, 52]],
 
        [[75, 76, 71],
         [72, 73, 68],
         [77, 78, 73],
         ...,
         [ 0,  0,  2],
         [ 3,  4,  6],
         [48, 49, 51]],
 
        ...,
 
        [[17, 18, 20],
         [ 3,  4,  6],
         [ 1,  2,  4],
         ...,
         [46, 57, 40],
         [46, 57, 40],
         [42, 53, 36]],
 
        [[21, 22, 24],
         [ 7,  8, 10],
         [ 4,  5,  7],
         ...,
         [70, 81, 64],
         [65, 76, 59],
         [57, 68, 51]],
 
        [[27, 29, 28],
         [12, 14, 13],
         [ 9, 11, 10],
         ...,
         [62, 75, 55],
         [56, 69, 49],
         [64, 77, 57]]], dtype=uint8),
 'elapsed_time': 0.10639452934265

### Batch Prediction

In [14]:
MODEL_PATH = "models-to-test/latest.pth"
MODEL_CONFIG_PATH = config_path= "mmdet_configs/xview_tood/tood_crop_300_500_cls_cars_trucks_1e-3_new_pipe.py"
EVAL_IMAGES_FOLDER_DIR = "warwickshire_images_resized/2020"
EXPORT_VISUAL = False

INFERENCE_SETTING_TO_PARAMS = {
    "XVIEW_SAHI": {
        "no_standard_prediction": True,
        "no_sliced_prediction": False,
        "slice_size": 400,
        "overlap_ratio": 0,
    },
    "XVIEW_SAHI_PO": {
        "no_standard_prediction": True,
        "no_sliced_prediction": False,
        "slice_size": 400,
        "overlap_ratio": 0.25,
    },
    "XVIEW_SAHI_FI": {
        "no_standard_prediction": False,
        "no_sliced_prediction": False,
        "slice_size": 400,
        "overlap_ratio": 0,
    },
    "XVIEW_SAHI_FI_PO": {
        "no_standard_prediction": False,
        "no_sliced_prediction": False,
        "slice_size": 400,
        "overlap_ratio": 0.25,
    },
}

# Of the above 4 options I want to slice and not do full image size inference
INFERENCE_SETTING = "XVIEW_SAHI_PO"
setting_params = INFERENCE_SETTING_TO_PARAMS[INFERENCE_SETTING]

# From the evaluation py file I delete the eval dataset path and change the confidence to 0.3
result = predict(
    model_type="mmdet",
    model_path=MODEL_PATH,
    model_config_path=MODEL_CONFIG_PATH,
    model_confidence_threshold=0.3,
    model_device="cuda:0",
    model_category_mapping=None,
    model_category_remapping=None,
    source=EVAL_IMAGES_FOLDER_DIR,
    no_standard_prediction=setting_params["no_standard_prediction"],
    no_sliced_prediction=setting_params["no_sliced_prediction"],
    image_size=None,
    slice_height=setting_params["slice_size"],
    slice_width=setting_params["slice_size"],
    overlap_height_ratio=setting_params["overlap_ratio"],
    overlap_width_ratio=setting_params["overlap_ratio"],
    postprocess_type="NMS",
    postprocess_match_metric="IOU",
    postprocess_match_threshold=0.5,
    postprocess_class_agnostic=False,
    novisual=not EXPORT_VISUAL,
    project="runs/predict_google_earth_mask",
    name=INFERENCE_SETTING,
    visual_bbox_thickness=None,
    visual_text_size=None,
    visual_text_thickness=None,
    visual_export_format="png",
    verbose=1,
    return_dict=True,
    force_postprocess_type=True,
    export_pickle = True,
)

There are 37 listed files in folder: 2020/
load checkpoint from local path: models-to-test/latest.pth


Performing inference on images:   0%|                                                        | 0/37 [00:00<?, ?it/s]

Performing prediction on 70 number of slices.


Performing inference on images:   3%|█▎                                              | 1/37 [00:06<04:03,  6.76s/it]

Prediction time is: 6688.13 ms
Performing prediction on 70 number of slices.


...

Prediction time is: 6768.45 ms
Performing prediction on 70 number of slices.


Performing inference on images:  97%|█████████████████████████████████████████████▋ | 36/37 [04:07<00:06,  6.93s/it]

Prediction time is: 6814.65 ms
Performing prediction on 70 number of slices.


Performing inference on images: 100%|███████████████████████████████████████████████| 37/37 [04:14<00:00,  6.87s/it]

Prediction time is: 6879.88 ms
Prediction results are successfully exported to runs/predict_google_earth_mask/XVIEW_SAHI_PO


### Putting it all together

In [15]:
df = pd.DataFrame(
   {
       "CP": int,
       "Day": [],
       "Month": [],
       "Year": [],
       "Small Car": [],
       "Bus": [],
       "Pickup Truck": [],
       "Utility Truck": [],
       "Truck": [],
       "Cargo Truck": [],
       "Truck w/Box": [],
       "Truck Tractor": [],
       "Trailer": [],
       "Truck w/Flatbed": [],
       "Truck w/Liquid": [],
       
   }
)

In [18]:
pickle_folder = 'runs/predict_google_earth_mask/XVIEW_SAHI_PO/pickles'
for i, file in enumerate(os.listdir(pickle_folder)):
    with (open(os.path.join(pickle_folder, file), "rb")) as f:
        prediction_list = pickle.load(f)
    
    image_name = file.split('.')[0]
    cp, date = image_name.split('_')
    cp = int(cp)
    month, day, year = date.split('-')
    counts = defaultdict(int)

    for pred in prediction_list:
        counts[pred.category.name] +=1

    df.loc[i] = [cp, 
                 day, 
                 month, 
                 year, 
                 counts["Small Car"],
                 counts["Bus"],
                 counts["Pickup Truck"],
                 counts["Utility Truck"],
                 counts["Truck"],
                 counts["Cargo Truck"],
                 counts["Truck w/Box"],
                 counts["Truck Tractor"],
                 counts["Trailer"],
                 counts["Truck w/Flatbed"],
                 counts["Truck w/Liquid"]
                ]

In [19]:
df

,CP,Day,Month,Year,Small Car,Bus,Pickup Truck,Utility Truck,Truck,Cargo Truck,Truck w/Box,Truck Tractor,Trailer,Truck w/Flatbed,Truck w/Liquid
0,73741,21,4,2020,6,6,0,1,5,6,6,0,1,0,0
1,8625,11,4,2020,9,3,0,3,3,0,0,0,0,0,0
2,8625,21,4,2020,11,0,0,0,3,3,3,0,0,0,0
3,89134,21,4,2020,2,0,0,0,0,0,0,0,0,0,0
4,58312,11,4,2020,3,3,0,0,0,0,4,0,0,0,0
5,84027,9,4,2020,14,2,0,0,2,1,1,0,0,0,0
6,58312,21,4,2020,2,1,0,0,0,0,2,0,0,0,0
7,73742,21,4,2020,76,8,0,1,6,2,1,0,0,0,0
8,7153,9,4,2020,7,4,0,0,0,1,4,0,0,0,0
9,73315,21,4,2020,38,4,0,0,3,2,1,0,0,0,0


We see an obvious problem, CPs with nearby car parks and residential areas are huge

In [20]:
# filtered_df = df.loc[df["Small Car"] <= 100]
# filtered_df

### DVLA Data Download

In [ ]:
! wget -O dft_traffic_counts_raw_counts.zip https://storage.googleapis.com/dft-statistics/road-traffic/downloads/data-gov-uk/dft_traffic_counts_raw_counts.zip
! unzip dft_traffic_counts_raw_counts.zip
! rm dft_traffic_counts_raw_counts.zip
! wget -O dft_traffic_counts_aadf.zip https://storage.googleapis.com/dft-statistics/road-traffic/downloads/data-gov-uk/dft_traffic_counts_aadf.zip
! unzip dft_traffic_counts_aadf.zip
! rm dft_traffic_counts_aadf.zip
! wget -O local_authority_traffic.csv https://storage.googleapis.com/dft-statistics/road-traffic/downloads/data-gov-uk/local_authority_traffic.csv
! rm __MACOSX
! ls

In [21]:
raw_counts = pd.read_csv("dft_traffic_counts_raw_counts.csv")
AADF = pd.read_csv("dft_traffic_counts_aadf.csv")
local_authority_traffic = pd.read_csv("local_authority_traffic.csv")

In [22]:
authority_names = local_authority_traffic[["local_authority_name", "local_authority_id"]]
AADF = AADF.merge(authority_names, on='local_authority_id', how='left')

In [31]:
AADF_ATC_2020_Warwickshire.drop_duplicates()

,id,count_point_id,year,region_id,local_authority_id,road_name,road_category,road_type,start_junction_road_name,end_junction_road_name,...,lgvs,hgvs_2_rigid_axle,hgvs_3_rigid_axle,hgvs_4_or_more_rigid_axle,hgvs_3_or_4_articulated_axle,hgvs_5_articulated_axle,hgvs_6_articulated_axle,all_hgvs,all_motor_vehicles,local_authority_name
1068300,39052,73741,2020,10,72,M42,TM,Major,10,LA Boundary,...,9529,1590,608,648,333,865,5093,9137,49929,Warwickshire
1104735,40387,99324,2020,10,72,A46,TA,Major,LA Boundary,A435 / A46,...,2351,726,75,66,84,185,1133,2269,16129,Warwickshire
1110645,40602,84026,2020,10,72,M42,TM,Major,M6 Toll and M42 Interchange North of Gilson - ...,M42 Junction 9,...,11011,2997,524,770,415,1232,7886,13825,64866,Warwickshire
1114537,40741,77205,2020,10,72,A5,TA,Major,B4116 Holly Lane,Kennel Lane,...,2435,561,152,157,172,443,363,1848,15399,Warwickshire
1119749,40929,58335,2020,10,72,A46,TA,Major,Stratford rd,A422,...,1719,398,73,65,68,409,246,1258,14018,Warwickshire
1120869,40969,73742,2020,10,72,A5,TA,Major,A47,LA Boundary,...,4724,612,159,277,130,492,790,2460,26322,Warwickshire
1122461,41027,57893,2020,10,72,M42,TM,Major,9,10,...,11449,3114,724,562,463,1554,6944,13361,64126,Warwickshire
1142332,41759,57624,2020,10,72,A5,TA,Major,A444,A47S,...,2772,478,110,175,203,739,376,2081,16142,Warwickshire
1190219,43516,73851,2020,10,72,M45,TM,Major,A45,LA Boundary,...,1560,301,53,86,88,668,278,1474,8770,Warwickshire
1190443,43524,36022,2020,10,72,M6,TM,Major,LA Boundary,1,...,8552,2997,446,208,500,2827,10553,17532,60291,Warwickshire


In [23]:
AADF_ATC_2020_Warwickshire = AADF.loc[(AADF.year == 2020) & (AADF.estimation_method_detailed == 'Automatic counter') & (AADF.local_authority_name == "Warwickshire")]
ATC_2020_Warwickshire = AADF_ATC_2020_Warwickshire.drop_duplicates().sort_values(by='road_name')[['count_point_id','cars_and_taxis', 'buses_and_coaches', 'lgvs', 'hgvs_2_rigid_axle', 'hgvs_3_rigid_axle','hgvs_4_or_more_rigid_axle','hgvs_3_or_4_articulated_axle','hgvs_5_articulated_axle','hgvs_6_articulated_axle','all_hgvs','all_motor_vehicles']]
ATC_2020_Warwickshire.head()

,count_point_id,cars_and_taxis,buses_and_coaches,lgvs,hgvs_2_rigid_axle,hgvs_3_rigid_axle,hgvs_4_or_more_rigid_axle,hgvs_3_or_4_articulated_axle,hgvs_5_articulated_axle,hgvs_6_articulated_axle,all_hgvs,all_motor_vehicles
1316907,7153,11916,32,3077,544,118,335,56,474,462,1989,17083
3858924,73315,24448,99,4727,1032,215,335,148,437,1071,3240,32634
1193169,56417,17876,89,3402,918,133,168,175,644,512,2550,23989
1382338,8625,29598,45,5470,1161,294,366,187,493,2100,4601,39822
1334751,38708,28351,61,5782,1651,232,452,171,622,1709,4835,39168


### Combine the two

In [25]:
ATC_2020_Warwickshire_train = df.merge(ATC_2020_Warwickshire, left_on='CP', right_on='count_point_id', how='left')

In [26]:
ATC_2020_Warwickshire_train

,CP,Day,Month,Year,Small Car,Bus,Pickup Truck,Utility Truck,Truck,Cargo Truck,...,buses_and_coaches,lgvs,hgvs_2_rigid_axle,hgvs_3_rigid_axle,hgvs_4_or_more_rigid_axle,hgvs_3_or_4_articulated_axle,hgvs_5_articulated_axle,hgvs_6_articulated_axle,all_hgvs,all_motor_vehicles
0,73741,21,4,2020,6,6,0,1,5,6,...,113,9529,1590,608,648,333,865,5093,9137,49929
1,8625,11,4,2020,9,3,0,3,3,0,...,45,5470,1161,294,366,187,493,2100,4601,39822
2,8625,21,4,2020,11,0,0,0,3,3,...,45,5470,1161,294,366,187,493,2100,4601,39822
3,89134,21,4,2020,2,0,0,0,0,0,...,1,38,5,1,0,1,3,5,14,351
4,58312,11,4,2020,3,3,0,0,0,0,...,101,6345,841,213,469,249,1910,811,4494,39603
5,84027,9,4,2020,14,2,0,0,2,1,...,8,978,274,32,82,46,385,271,1090,6878
6,58312,21,4,2020,2,1,0,0,0,0,...,101,6345,841,213,469,249,1910,811,4494,39603
7,73742,21,4,2020,76,8,0,1,6,2,...,142,4724,612,159,277,130,492,790,2460,26322
8,7153,9,4,2020,7,4,0,0,0,1,...,32,3077,544,118,335,56,474,462,1989,17083
9,73315,21,4,2020,38,4,0,0,3,2,...,99,4727,1032,215,335,148,437,1071,3240,32634


In [27]:
ATC_2020_Warwickshire_train.to_csv (r'ATC_2020_Warwickshire_train.csv', index = False, header=True)